In [1]:
# using CuArrays
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

┌ Info: Recompiling stale cache file /Users/mertceylan/.julia/compiled/v1.1/Deeplearning/gtjFf.ji for Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de]
└ @ Base loading.jl:1184
┌ Warning: Package Deeplearning does not have LinearAlgebra in its dependencies:
│ - If you have Deeplearning checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Deeplearning
└ Loading LinearAlgebra into Deeplearning from project dependency, future warnings for Deeplearning are suppressed.


In [6]:
batch = 100
learning_rate = 10e-2

0.1

In [7]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []
for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end
println("Ready...")

Ready...


In [11]:
conv1 = Deeplearning.ConvolutionLayer((3,3,6), (2,2), (1,1), randn, relu)((28,28,1,1))
conv2 = Deeplearning.ConvolutionLayer((3,3,16), (2,2), (1,1), randn, relu)(conv1.telemetry.o)
dense1 = Deeplearning.FullyConnectedLayer(120, randn, relu)(conv2.telemetry.o)
dense2 = Deeplearning.FullyConnectedLayer(84, randn, relu)(dense1.telemetry.o)
dense3 = Deeplearning.FullyConnectedLayer(10, randn, relu)(dense2.telemetry.o)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

Loaded


In [12]:
function f(x)
    x = conv1(x)    
    x = conv2(x)    
    x = dense1(x)
    x = dense2(x)
    x = dense3(x)
    x = softmax(x)
end

f (generic function with 1 method)

In [ ]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

In [13]:
indices = sample(1:size(train)[1], size(train)[1], replace = false)

42000-element Array{Int64,1}:
  6665
 35234
 28678
  2557
 17512
 23414
  5065
 28863
   521
  6787
 11420
 32255
 39855
     ⋮
 33503
  8228
 25024
 17487
 23994
 36074
 39046
   164
  2989
 18699
 21795
  9589

In [14]:
sample_batch_x, sample_batch_y = cat(xtrain[indices[(1):(100)]]..., dims=4), ytrain[indices[(1):(100)]]

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Any[4, 6, 6, 4, 9, 8, 7, 9, 10, 8  …  9, 4, 7, 10, 5, 6, 2, 10, 4, 6])

In [15]:
dff(x,y) = @diff nll(f(x), y)
dv = dff(sample_batch_x, sample_batch_y)

T(0.0801013746258996)

In [ ]:
dff(x,y) = @diff nll(f(x), y)

for e in 1:1
    println("Epoch ", e)
    indices = sample(1:size(train)[1], size(train)[1], replace = false)
    for (ei,si) in enumerate(1:batch:size(indices)[1])
        sample_batch_x, sample_batch_y = cat(xtrain[indices[(si):(ei*batch)]]..., dims=4), ytrain[indices[(si):(ei*batch)]]

        dv = dff(sample_batch_x, sample_batch_y)

        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            layers[pidx].matrix .-=0.1*dw
        end
    end
    otrain = f(xtrain)
#     otest = f(xtest)
    push!(acc_train_plot, acc(otrain, ytrain))
    push!(loss_train_plot, nll(otrain, ytrain))
#     push!(acc_test_plot, acc(o, ytest))
#     push!(loss_test_plot, nll(o, ytest))
end

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")